In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from sympy import symbols, solve
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display


## Importing data

In [2]:
df_data = pd.read_excel('data_with_mean_ESG.xlsx', header=[0,1], index_col=0)

## Filtering data for a certain sustianable threshold

In [3]:
def filter_esg_stocks(df, score_type, threshold=None):
    valid_score_type = ['ESG', 'Enviromental', 'Social', 'Governance']
    if score_type not in valid_score_type:
        raise ValueError("Invalid score type. Please choose from ESG, Enviromental, Social or Governance")

    # Filter dataframe based on threshol
    if threshold is not None:
        filtered = df[df[score_type + ' Disc Mean'][score_type + ' Disc mean score'] >= threshold]

    return filtered



    

In [4]:
filtered = filter_esg_stocks(df_data, 'ESG', threshold=60)
filtered

Last Price                                          \
Tickers 2006-12-31 00:00:00 2007-12-31 00:00:00 2008-12-31 00:00:00   
SHEL                  34.93            41.85480            26.30788   
BP/                   11.11            12.19361             7.66645   
JNJ                   66.02            66.70000            59.83000   
BAC                   53.39            41.26000            14.08000   

                                                                     \
Tickers 2009-12-31 00:00:00 2010-12-31 00:00:00 2011-12-30 00:00:00   
SHEL               30.39054            33.34135            36.77184   
BP/                 9.68880             7.25839             7.14189   
JNJ                64.41000            61.85000            65.58000   
BAC                15.06000            13.34000             5.56000   

                                                                     \
Tickers 2012-12-31 00:00:00 2013-12-31 00:00:00 2014-12-31 00:00:00   
SHEL               34.46552            35.83226            33.55368   
BP/                 6.89960             8.08504             6.40379   
JNJ                70.10000            91.59000           104.57000   
BAC                11.61000            15.57000            17.89000   

                             ... Governance Disclosure                      \
Tickers 2015-12-31 00:00:00  ...   2016-12-30 00:00:00 2017-12-29 00:00:00   
SHEL               22.48408  ...               91.2402             91.2402   
BP/                 5.21584  ...               91.2402             91.2402   
JNJ               102.72000  ...               93.7387             93.7387   
BAC                16.83000  ...               93.7387             93.7387   

                                                                     \
Tickers 2018-12-31 00:00:00 2019-12-31 00:00:00 2020-12-31 00:00:00   
SHEL                91.2402             91.2402             91.2402   
BP/                 91.2402             91.2402             86.2432   
JNJ                 93.7400             93.7400             93.7400   
BAC                 93.7387             93.7387             93.7387   

                                  ESG Disc Mean       Enviromental Disc Mean  \
Tickers 2021-12-31 00:00:00 ESG Disc mean score Enviromental Disc mean score   
SHEL                91.2402           60.451663                    47.383719   
BP/                 86.2432           61.297900                    44.399350   
JNJ                 93.7400           62.538137                    57.280975   
BAC                100.0000           61.070312                    55.812081   

              Social Disc Mean       Governance Disc Mean  
Tickers Social Disc mean score Governance Disc mean score  
SHEL                 46.623725                  87.246019  
BP/                  50.402425                  88.990081  
JNJ                  41.030206                  89.197294  
BAC                  33.460550                  93.804569  

[4 rows x 84 columns]

In [5]:
labels = filtered.index
labels

Index(['SHEL ', 'BP/ ', 'JNJ ', 'BAC '], dtype='object')

## Stock selection

In [6]:
def create_portfolio(df, labels):
    """
    Function to create a stock portfolio by choosing stocks from a dataframe.

    Args:
        df (pd.DataFrame): DataFrame containing stock tickers, last prices, and ESG scores.

    Returns:
        pd.DataFrame: DataFrame containing the selected stocks and their respective variables and scores.
    """
    # Get unique stock tickers from the dataframe
    stock_tickers = labels.unique()

    # Create widgets for user inputs
    num_stocks_widget = widgets.IntSlider(value=1, min=1, max=len(stock_tickers), step=1, description='Number of stocks:')
    stock_dropdowns = [widgets.Dropdown(options=stock_tickers, description=f'Stock {i+1}:') for i in range(num_stocks_widget.value)]
    exit_button = widgets.Button(description='Exit')
    display(num_stocks_widget)
    for dropdown in stock_dropdowns:
        display(dropdown)
    display(exit_button)

    # Define function to update dropdowns based on number of stocks selected
    def update_dropdowns(change):
        num_stocks = num_stocks_widget.value
        for i in range(num_stocks):
            if len(stock_dropdowns) < num_stocks:
                dropdown = widgets.Dropdown(options=stock_tickers, description=f'Stock {i+1}:')
                stock_dropdowns.append(dropdown)
            else:
                stock_dropdowns[i].options = stock_tickers[:]

    # Add event listener to update dropdowns when number of stocks changes
    num_stocks_widget.observe(update_dropdowns, names='value')

    # Define function to handle exit button click
    def exit_button_clicked(button):
        nonlocal exit_flag
        exit_flag = True

    # Add event listener to handle exit button click
    exit_flag = False
    exit_button.on_click(exit_button_clicked)

    # Wait for user input
    print("Please select stocks from the dropdowns above, or click 'Exit' to finish:")
    stocks_selected = []
    while not exit_flag:
        stocks_selected = [dropdown.value for dropdown in stock_dropdowns]

    # Create a new dataframe to store selected stocks and their respective variables and scores
    selected_df = df[df['Ticker'].isin(stocks_selected)].reset_index(drop=True)

    return selected_df

In [7]:
def create_portfolio1(dataframe):
    """
    Function to interactively create a portfolio of stocks from a DataFrame.

    Args:
        dataframe (pd.DataFrame): DataFrame containing stock tickers, last prices, and ESG scores.

    Returns:
        pd.DataFrame: DataFrame containing selected stocks with their respective variables and scores.
    """
    portfolio = pd.DataFrame(columns=dataframe.columns)

    # Create widgets for user input
    num_stocks_widget = widgets.IntSlider(
        value=1,
        min=1,
        max=len(dataframe),
        step=1,
        description='Number of stocks:',
        continuous_update=False
    )
    stock_dropdown_widget = widgets.Dropdown(
        options=dataframe.index,
        description='Stocks:',
        continuous_update=False
    )
    add_button = widgets.Button(description='Add stock')
    exit_button = widgets.Button(description='Exit')
    
    # Define button click event handlers
    def add_stock(button):
        stock_ticker = stock_dropdown_widget.value
        stock_data = dataframe.loc[stock_ticker]
        portfolio.loc[stock_ticker] = stock_data
        print(f'Stock {stock_ticker} added to portfolio.')

    def exit(button):
        display(portfolio)
        return

    # Assign event handlers to buttons
    add_button.on_click(add_stock)
    exit_button.on_click(exit)

    # Display widgets
    display(num_stocks_widget)
    display(stock_dropdown_widget)
    display(add_button)
    display(exit_button)

    return portfolio


In [8]:
selected = create_portfolio1(filtered)


IntSlider(value=1, continuous_update=False, description='Number of stocks:', max=4, min=1)

Dropdown(description='Stocks:', options=('SHEL ', 'BP/ ', 'JNJ ', 'BAC '), value='SHEL ')

Button(description='Add stock', style=ButtonStyle())

Button(description='Exit', style=ButtonStyle())

Stock BP/  added to portfolio.
Stock JNJ  added to portfolio.


Last Price                                          \
Tickers 2006-12-31 00:00:00 2007-12-31 00:00:00 2008-12-31 00:00:00   
BP/                   11.11            12.19361             7.66645   
JNJ                   66.02            66.70000            59.83000   

                                                                     \
Tickers 2009-12-31 00:00:00 2010-12-31 00:00:00 2011-12-30 00:00:00   
BP/                  9.6888             7.25839             7.14189   
JNJ                 64.4100            61.85000            65.58000   

                                                                     \
Tickers 2012-12-31 00:00:00 2013-12-31 00:00:00 2014-12-31 00:00:00   
BP/                  6.8996             8.08504             6.40379   
JNJ                 70.1000            91.59000           104.57000   

                             ... Governance Disclosure                      \
Tickers 2015-12-31 00:00:00  ...   2016-12-30 00:00:00 2017-12-29 00:00:00   
BP/                 5.21584  ...               91.2402             91.2402   
JNJ               102.72000  ...               93.7387             93.7387   

                                                                     \
Tickers 2018-12-31 00:00:00 2019-12-31 00:00:00 2020-12-31 00:00:00   
BP/                 91.2402             91.2402             86.2432   
JNJ                 93.7400             93.7400             93.7400   

                                  ESG Disc Mean       Enviromental Disc Mean  \
Tickers 2021-12-31 00:00:00 ESG Disc mean score Enviromental Disc mean score   
BP/                 86.2432           61.297900                    44.399350   
JNJ                 93.7400           62.538137                    57.280975   

              Social Disc Mean       Governance Disc Mean  
Tickers Social Disc mean score Governance Disc mean score  
BP/                  50.402425                  88.990081  
JNJ                  41.030206                  89.197294  

[2 rows x 84 columns]

Stock SHEL  added to portfolio.


Last Price                                          \
Tickers 2006-12-31 00:00:00 2007-12-31 00:00:00 2008-12-31 00:00:00   
BP/                   11.11            12.19361             7.66645   
JNJ                   66.02            66.70000            59.83000   
SHEL                  34.93            41.85480            26.30788   

                                                                     \
Tickers 2009-12-31 00:00:00 2010-12-31 00:00:00 2011-12-30 00:00:00   
BP/                 9.68880             7.25839             7.14189   
JNJ                64.41000            61.85000            65.58000   
SHEL               30.39054            33.34135            36.77184   

                                                                     \
Tickers 2012-12-31 00:00:00 2013-12-31 00:00:00 2014-12-31 00:00:00   
BP/                 6.89960             8.08504             6.40379   
JNJ                70.10000            91.59000           104.57000   
SHEL               34.46552            35.83226            33.55368   

                             ... Governance Disclosure                      \
Tickers 2015-12-31 00:00:00  ...   2016-12-30 00:00:00 2017-12-29 00:00:00   
BP/                 5.21584  ...               91.2402             91.2402   
JNJ               102.72000  ...               93.7387             93.7387   
SHEL               22.48408  ...               91.2402             91.2402   

                                                                     \
Tickers 2018-12-31 00:00:00 2019-12-31 00:00:00 2020-12-31 00:00:00   
BP/                 91.2402             91.2402             86.2432   
JNJ                 93.7400             93.7400             93.7400   
SHEL                91.2402             91.2402             91.2402   

                                  ESG Disc Mean       Enviromental Disc Mean  \
Tickers 2021-12-31 00:00:00 ESG Disc mean score Enviromental Disc mean score   
BP/                 86.2432           61.297900                    44.399350   
JNJ                 93.7400           62.538137                    57.280975   
SHEL                91.2402           60.451663                    47.383719   

              Social Disc Mean       Governance Disc Mean  
Tickers Social Disc mean score Governance Disc mean score  
BP/                  50.402425                  88.990081  
JNJ                  41.030206                  89.197294  
SHEL                 46.623725                  87.246019  

[3 rows x 84 columns]

Stock SHEL  added to portfolio.


Last Price                                          \
Tickers 2006-12-31 00:00:00 2007-12-31 00:00:00 2008-12-31 00:00:00   
BP/                   11.11            12.19361             7.66645   
JNJ                   66.02            66.70000            59.83000   
SHEL                  34.93            41.85480            26.30788   

                                                                     \
Tickers 2009-12-31 00:00:00 2010-12-31 00:00:00 2011-12-30 00:00:00   
BP/                 9.68880             7.25839             7.14189   
JNJ                64.41000            61.85000            65.58000   
SHEL               30.39054            33.34135            36.77184   

                                                                     \
Tickers 2012-12-31 00:00:00 2013-12-31 00:00:00 2014-12-31 00:00:00   
BP/                 6.89960             8.08504             6.40379   
JNJ                70.10000            91.59000           104.57000   
SHEL               34.46552            35.83226            33.55368   

                             ... Governance Disclosure                      \
Tickers 2015-12-31 00:00:00  ...   2016-12-30 00:00:00 2017-12-29 00:00:00   
BP/                 5.21584  ...               91.2402             91.2402   
JNJ               102.72000  ...               93.7387             93.7387   
SHEL               22.48408  ...               91.2402             91.2402   

                                                                     \
Tickers 2018-12-31 00:00:00 2019-12-31 00:00:00 2020-12-31 00:00:00   
BP/                 91.2402             91.2402             86.2432   
JNJ                 93.7400             93.7400             93.7400   
SHEL                91.2402             91.2402             91.2402   

                                  ESG Disc Mean       Enviromental Disc Mean  \
Tickers 2021-12-31 00:00:00 ESG Disc mean score Enviromental Disc mean score   
BP/                 86.2432           61.297900                    44.399350   
JNJ                 93.7400           62.538137                    57.280975   
SHEL                91.2402           60.451663                    47.383719   

              Social Disc Mean       Governance Disc Mean  
Tickers Social Disc mean score Governance Disc mean score  
BP/                  50.402425                  88.990081  
JNJ                  41.030206                  89.197294  
SHEL                 46.623725                  87.246019  

[3 rows x 84 columns]

In [10]:
selected

Last Price                                          \
Tickers 2006-12-31 00:00:00 2007-12-31 00:00:00 2008-12-31 00:00:00   
BP/                   11.11            12.19361             7.66645   
JNJ                   66.02            66.70000            59.83000   
SHEL                  34.93            41.85480            26.30788   

                                                                     \
Tickers 2009-12-31 00:00:00 2010-12-31 00:00:00 2011-12-30 00:00:00   
BP/                 9.68880             7.25839             7.14189   
JNJ                64.41000            61.85000            65.58000   
SHEL               30.39054            33.34135            36.77184   

                                                                     \
Tickers 2012-12-31 00:00:00 2013-12-31 00:00:00 2014-12-31 00:00:00   
BP/                 6.89960             8.08504             6.40379   
JNJ                70.10000            91.59000           104.57000   
SHEL               34.46552            35.83226            33.55368   

                             ... Governance Disclosure                      \
Tickers 2015-12-31 00:00:00  ...   2016-12-30 00:00:00 2017-12-29 00:00:00   
BP/                 5.21584  ...               91.2402             91.2402   
JNJ               102.72000  ...               93.7387             93.7387   
SHEL               22.48408  ...               91.2402             91.2402   

                                                                     \
Tickers 2018-12-31 00:00:00 2019-12-31 00:00:00 2020-12-31 00:00:00   
BP/                 91.2402             91.2402             86.2432   
JNJ                 93.7400             93.7400             93.7400   
SHEL                91.2402             91.2402             91.2402   

                                  ESG Disc Mean       Enviromental Disc Mean  \
Tickers 2021-12-31 00:00:00 ESG Disc mean score Enviromental Disc mean score   
BP/                 86.2432           61.297900                    44.399350   
JNJ                 93.7400           62.538137                    57.280975   
SHEL                91.2402           60.451663                    47.383719   

              Social Disc Mean       Governance Disc Mean  
Tickers Social Disc mean score Governance Disc mean score  
BP/                  50.402425                  88.990081  
JNJ                  41.030206                  89.197294  
SHEL                 46.623725                  87.246019  

[3 rows x 84 columns]

## ESG Score that investor wants

### Mean esg score

In [1005]:
esg = df_data['ESG Disclosure'].iloc[17:24,4:16]
esg

Tickers  2010-12-31  2011-12-30  2012-12-31  2013-12-31  2014-12-31  \
PCG         61.3520     60.4064     55.3767     55.9602     56.5335   
GOOGL       30.4597     33.6284     35.0367     35.8415     35.8415   
GOOG        30.4597     33.6284     35.0367     35.8415     35.8415   
VOW         39.0102     39.5332     41.4948     43.9996     44.5227   
ALV         33.6385     37.8030     38.7486     51.1619     52.5500   
ADS         43.4564     39.1208     50.5080     47.9529     52.8317   
RSG         29.6550     29.6550     30.4899     33.0148     36.8675   

Tickers  2015-12-31  2016-12-30  2017-12-29  2018-12-31  2019-12-31  \
PCG         61.2715     61.6135     65.4964     63.9775     58.7567   
GOOGL       43.2250     51.3932     55.9099     56.4330     58.6862   
GOOG        43.2250     51.3932     55.9099     56.4330     58.6862   
VOW         54.2199     53.3850     62.4887     62.2372     62.2372   
ALV         56.1815     56.5436     56.3525     59.1490     60.0845   
ADS         54.4613     51.7453     55.2460     55.5679     55.5679   
RSG         43.8185     47.0878     55.3164     55.3164     56.3625   

Tickers  2020-12-31  2021-12-31  
PCG         67.4278     64.4503  
GOOGL       58.6862     59.7827  
GOOG        58.6862     59.7827  
VOW         62.9011     63.1727  
ALV         60.4265     57.4992  
ADS         58.6058     58.7064  
RSG         62.2473     69.7314

In [1006]:
mean_esg = esg.mean(axis=1)
mean_esg

PCG       61.051875
GOOGL     46.243667
GOOG      46.243667
VOW       52.433525
ALV       51.678233
ADS       51.980867
RSG       45.796875
dtype: float64

In [1007]:
esg_np = mean_esg.to_numpy()
esg_np

array([61.051875  , 46.24366667, 46.24366667, 52.433525  , 51.67823333,
       51.98086667, 45.796875  ])

## Log returns

In [1008]:
from log_return import log_returns
log_re = log_returns(stocks).replace([np.inf, -np.inf], np.nan).dropna(axis=1)

In [1009]:
log_return = log_re.T
log_return

PCG     GOOGL      GOOG       VOW       ALV       ADS   \
Tickers                                                                  
2011-12-30 -0.148939  0.083782  0.083866 -0.044586 -0.208112  0.004526   
2012-12-31 -0.025554  0.090953  0.091026  0.471210  0.369220  0.312411   
2013-12-31  0.002486  0.460133  0.460067  0.233267  0.260956  0.361902   
2014-12-31  0.278955 -0.055421 -0.066042 -0.215947 -0.074454 -0.601615   
2015-12-31 -0.000940  0.382631  0.368522 -0.343708  0.066453  0.336806   
2016-12-30  0.133226  0.018390  0.016908 -0.073713 -0.074803  0.478896   
2017-12-29 -0.304219  0.284636  0.304360  0.340864  0.329539  0.238154   
2018-12-31 -0.635295 -0.008044 -0.010355 -0.242464 -0.138837  0.037776   
2019-12-31 -0.781576  0.248243  0.255443  0.199235  0.200435  0.442681   
2020-12-31  0.136517  0.268901  0.270246  0.073460  0.007292  0.119376   
2021-12-31 -0.026018  0.502566  0.501812  0.335547 -0.048533 -0.245153   

                RSG   
Tickers               
2011-12-30 -0.080517  
2012-12-31  0.062608  
2013-12-31  0.123939  
2014-12-31  0.192560  
2015-12-31  0.088852  
2016-12-30  0.259966  
2017-12-29  0.169828  
2018-12-31  0.064159  
2019-12-31  0.217775  
2020-12-31  0.071778  
2021-12-31  0.370238

In [1010]:
log_return.index

DatetimeIndex(['2011-12-30', '2012-12-31', '2013-12-31', '2014-12-31',
               '2015-12-31', '2016-12-30', '2017-12-29', '2018-12-31',
               '2019-12-31', '2020-12-31', '2021-12-31'],
              dtype='datetime64[ns]', name='Tickers', freq=None)

In [1011]:
fig = px.line(log_return, x=log_return.index, y=log_return.columns, title='Logarithmic Returns of stocks ')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Log Return in %')
fig.show()

## Labels

In [1012]:
labels = log_re.index
labels

Index(['PCG ', 'GOOGL ', 'GOOG ', 'VOW ', 'ALV ', 'ADS ', 'RSG '], dtype='object')

## Calculating Covariance matrix

In [1013]:
from covariance import cov_matrix
cov_var = cov_matrix(log_re.T)

## Inverse of covariance matrix

In [1014]:
inv_cov = np.linalg.inv(cov_var)

## Mean return of each stock

In [1015]:
from mean_ret import mean_return
mean_re = mean_return(log_re.T)

## Vector of ones

In [1016]:
from ones import vector_ones
vector_one = vector_ones(labels)

## Calculate MVP

### useful equations are calculated for us to derive the minimum portfolio

In [1017]:
from mvp import mvp_func
a, b, c, d = mvp_func(inv_cov, mean_re, vector_one)

In [1018]:
a

12.038315392864833

### Finding the optimal weights for minimal variance portfolio, with two different expected returns

In [1019]:
from weights_mvp import mvp_weights
w1, w2 = mvp_weights(a, b, c, d, inv_cov, vector_one, mean_re, 0.22, 0.28)

In [1020]:
w1

array([-0.19025461,  6.36298082, -5.98534045, -0.03445611, -0.02211064,
        0.09521903,  0.77396195])

In [1021]:
w2

array([-0.3356703 ,  8.44563561, -7.88569103, -0.04501748, -0.12992395,
        0.0910686 ,  0.85959855])

### Creating a list that gives values between any given value inside a list

In [1022]:
from alpha import alpha
alpha_list = alpha([-5])

## We can now use the two fund theorem to find the weights for the entire effecient frontier, using two optimal solutions

In [1023]:
from two_fun_theorem import w3
ef_weights = w3(w1, w2, alpha_list)

In [1024]:
ef_weights

array([[ -1.06274878,  18.85890959, -17.38744397, ...,  -0.66899051,
          0.07031643,   1.28778158],
       [ -1.06129462,  18.83808304, -17.36844046, ...,  -0.66791238,
          0.07035794,   1.28692521],
       [ -1.05984047,  18.8172565 , -17.34943696, ...,  -0.66683424,
          0.07039944,   1.28606884],
       ...,
       [  0.38849986,  -1.92598527,   1.57805489, ...,   0.40698635,
          0.11173775,   0.43312826],
       [  0.38995402,  -1.94681181,   1.5970584 , ...,   0.40806448,
          0.11177926,   0.43227189],
       [  0.39140818,  -1.96763836,   1.6160619 , ...,   0.40914261,
          0.11182076,   0.43141553]])

## Calculating the expected return for each portfolio on the effecient frontier

In [1025]:
from expected_return import exp_return
r = exp_return(ef_weights, mean_re)

In [1047]:
r

array([ 0.58  ,  0.5794,  0.5788, ..., -0.0188, -0.0194, -0.02  ])

## Calculating variance and volatility for each portfolio on the effecient frontier

#### Variance

In [1026]:
from variance import var_portfolio
port_variance = var_portfolio(ef_weights, cov_var)

In [1049]:
port_variance

array([0.1951357 , 0.19466352, 0.19419195, ..., 0.02204817, 0.02217408,
       0.02230058])

#### Volatility

In [1027]:
from volatility import volatility
vol = volatility(port_variance)

In [1048]:
vol

array([0.44174167, 0.44120689, 0.44067215, ..., 0.14848627, 0.14890963,
       0.1493338 ])

## labels for the plot

In [1028]:
tickers_list = []
for i in range(len(ef_weights)):
    #assets = np.random.choice(list(labels), len(labels), replace=False)
    assets = list(labels)
    tickers_list.append(assets)

## Plot of the effecient frontier

In [1029]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=vol, y=r,
                        mode='lines',
                        line=go.scatter.Line(color="orange"),
                        showlegend=True,
                        name='Portfolios',
                        text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(ef_weights[i]).round(2)) for i in range(len(tickers_list))]))

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1    
    ))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Volatility'),
                  yaxis=dict(title='Expected Return'),
                  width=1000,
                  height=600)

fig.show()

## SR

### Calculating weights of SR

In [1030]:
#Assuming a risk free rate of 0% as an example
sr = r / vol

In [1031]:
weights_sr = ef_weights[sr.argmax()]
weights_sr

array([-1.58263152e-01,  5.90479676e+00, -5.56726332e+00, -3.21326066e-02,
        1.60829195e-03,  9.61321261e-02,  7.55121893e-01])

### SR tickers list and weights

In [1032]:
sr_t_list = []
for i in range(len(weights_sr)):
    assets = list(labels)
    sr_t_list.append(assets)

### Calculating return and volatility of SR

In [1033]:
#r_sr = exp_return(weights_sr, mean_re)
sr_r = weights_sr@mean_re

In [1034]:
sr_r

0.20679999999989168

In [1035]:
#variance_sr = var_portfolio(weights_sr, cov_var)
var_sr = weights_sr@cov_var@weights_sr
var_sr

0.01718189655918986

In [1036]:
vol_sr = np.sqrt(var_sr)

In [1037]:
vol_sr

0.1310797335944419

## MVP

In [1038]:
weights_mvp = ef_weights[vol.argmin()]
weights_mvp

array([ 0.0831269 ,  2.4475898 , -2.41268134, -0.01460073,  0.18057839,
        0.10302185,  0.61296513])

### Weights and tickers for MVP

In [1039]:
mvp_t_list = []
for i in range(len(weights_mvp)):
    assets = list(labels)
    mvp_t_list.append(assets)

### Volatility and return of MVP

In [1040]:
mvp_r = weights_mvp@mean_re
mvp_r

0.10719999999991783

In [1041]:
#variance_sr = var_portfolio(weights_sr, cov_var)
var_mvp = weights_mvp@cov_var@weights_mvp
var_sr

0.01718189655918986

In [1042]:
vol_mvp = np.sqrt(var_mvp)

In [1043]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=vol, y=r,
                        mode='lines',
                        line=go.scatter.Line(color="orange"),
                        showlegend=True,
                        name='Portfolios',
                        text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(ef_weights[i]).round(2)) for i in range(len(tickers_list))]))

fig.add_trace(go.Scatter(x = [vol_sr], y = [sr_r],
                         marker=dict(color='red', size=12), 
                         mode='markers',
                         name='SR',
                         #text=['SR'],
                         textposition="bottom center",
                         text=[str(np.array(sr_t_list[j])) + "<br>" + str(np.array(ef_weights[j]).round(2)) for j in range(len(sr_t_list))]))

fig.add_trace(go.Scatter(x = [vol_mvp], y = [mvp_r],
                         marker=dict(color='black', size=12), 
                         mode='markers',
                         name='MVP',
                         textposition="bottom center",
                         #text=['MVP'],
                         text=[str(np.array(mvp_t_list[k])) + "<br>" + str(np.array(ef_weights[k]).round(2)) for k in range(len(mvp_t_list))]))

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1    
    ))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Volatility'),
                  yaxis=dict(title='Expected Return'),
                  width=1000,
                  height=600)

fig.show()

## Weighted average esg score in a portfolio

In [1044]:
asset = labels.to_numpy()
asset

array(['PCG ', 'GOOGL ', 'GOOG ', 'VOW ', 'ALV ', 'ADS ', 'RSG '],
      dtype=object)

In [1045]:
from weighted_average import aw_esg
weighted_av = aw_esg(esg_np, ef_weights, asset)
weighted_av

array([26.09311835, 26.12178546, 26.15045258, ..., 54.70289946,
       54.73156657, 54.76023369])

In [1046]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=vol, y=r,
                        mode='lines',
                        line=go.scatter.Line(color="orange"),
                        showlegend=True,
                        name='Portfolios',
                        #text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(ef_weights[i]).round(2)) for i in range(len(tickers_list))]))
                        text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(ef_weights[i]).round(2)) + "<br>" + str(weighted_av[i])  for i in range(len(ef_weights))  ]))

fig.add_trace(go.Scatter(x = [vol_sr], y = [sr_r],
                         marker=dict(color='red', size=12), 
                         mode='markers',
                         name='SR',
                         #text=['SR'],
                         textposition="bottom center",
                         text=[str(np.array(sr_t_list[j])) + "<br>" + str(np.array(ef_weights[j]).round(2)) for j in range(len(sr_t_list))]))

fig.add_trace(go.Scatter(x = [vol_mvp], y = [mvp_r],
                         marker=dict(color='black', size=12), 
                         mode='markers',
                         name='MVP',
                         textposition="bottom center",
                         #text=['MVP'],
                         text=[str(np.array(mvp_t_list[k])) + "<br>" + str(np.array(ef_weights[k]).round(2)) for k in range(len(mvp_t_list))]))

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1    
    ))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Volatility'),
                  yaxis=dict(title='Expected Return'),
                  width=1000,
                  height=600)

fig.show()